In [1]:

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords

# global variables
bucket = "6893_bigdata_isaac"
output_directory_hashtags = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/hashtagsCount'.format(bucket)
output_directory_wordcount = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/wordcount'.format(bucket)
output_directory_canditweet = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/canditweet'.format(bucket)
output_directory_topwords = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/top5words'.format(bucket)
bucket_directory = 'gs://{}/largescale_data_processing/6889project/testdata6889.csv'.format(bucket)

# output table and columns name
output_dataset = 'bigdata_sparkStreaming'
output_table_hashtags = 'hashtags'
columns_name_hashtags = ['hashtags', 'count']
output_table_wordcount = 'wordcount'
columns_name_wordcount = ['word', 'count', 'time']
output_table_canditweet = 'canditweet'
columns_name_canditweet = ['candidate', 'tweet', 'time']
output_table_topwords = 'top5words'
columns_name_topwords = ['candidate','word','count','time']
# parameter
IP = 'localhost'
PORT = 9001

STREAMTIME = 60
STEP = int(STREAMTIME)/6

WORD = ['trump', 'nikki haley', 'ramaswamy', 'asa hutchinson', 'biden', 'marianne williamson']
#candidates = ['donald trump', 'nikki haley', 'vivek ramaswamy', 'asa hutchinson', 'joe biden', 'marianne williamson']

#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
common_word = ['donald', 'trump', 'nikki', 'haley', 'vivek', 'ramaswamy', 
               'asa', 'hutchinson', 'joe', 'biden', 'marianne', 'williamson', 
               'president', 'election', 'people']

# Helper functions
def saveToStorage(rdd, output_directory, columns_name, mode):
    
    if not rdd.isEmpty():
        (rdd.toDF( columns_name ) \
        .write.save(output_directory, format="json", mode=mode))
    #else:
    #    print('nothing in rdd {}'.format(output_directory))


def saveToBigQuery(sc, output_dataset, output_table, directory):
    
    files = directory + '/part-*'
    subprocess.check_call(
        'bq load --source_format NEWLINE_DELIMITED_JSON '
        '--replace '
        '--autodetect '
        '{dataset}.{table} {files}'.format(
            dataset=output_dataset, table=output_table, files=files
        ).split())
    output_path = sc._jvm.org.apache.hadoop.fs.Path(directory)
    output_path.getFileSystem(sc._jsc.hadoopConfiguration()).delete(
        output_path, True)
    
def saveToBucket(rdd, bucket_path):
    
    df = rdd.toDF(['candidate', 'tweet', 'time'])
    df.write.format("csv").option("header", "true").save(bucket_path)
    

def topWordCount(tweet):
    """
    input: dstream of format (candidate, related_tweet)
    output: transformed DStream in format (candidate, word, count, time)
    
    The function takes in DStream of candidate and its correlated tweets and returns the top word count dstream data in window
    """
    #stack every tweet in the time period in one line string
    grouped_tweets = tweet.reduceByKey(lambda x, y: x + ' ' + y)
    #grouped_tweets.pprint()
    #filter out the nltk stop words and self defined common words
    candi_word = grouped_tweets.map(lambda x: (x[0], x[1].split())) \
                    .map(lambda x: (x[0], [w for w in x[1] if w not in stop_words and w not in common_word]))
    candi_word.pprint()
    #flatmap to (candidate, word, 1) and doing reduceByKeyAndWindow
    candi_allwords_flat = candi_word.flatMap(lambda x: [((x[0], words), 1) for words in x[1]]) \
                            .reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 10, 10)
    candi_allwords_flat.pprint()
    #group the data and keep the top word
    top_words = candi_allwords_flat.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                                   .groupByKey() \
                                   .mapValues(lambda x: sorted(x, key=lambda y: y[1], reverse=True)[0])
    top_words.pprint()
    #map to extract the top word for each candidate
    most_count_words = top_words.map(lambda x: (x[0], x[1][0], x[1][1]))
    most_count_words.pprint()
    #map with the time that this wordcount is been recorded
    word_total = most_count_words.transform(lambda time, rdd: \
                                            rdd.map(lambda x: (x[0], x[1], x[2], time.strftime("%Y-%m-%d %H:%M:%S"))))
    word_total.pprint()
    
    return word_total
    

def canditweet(tweet):
    candiTweets_total = tweet.transform(lambda time, rdd: rdd \
                                        .map(lambda x: (x[0], SentimentIntensityAnalyzer().polarity_scores(x[1])['compound'], time.strftime("%Y-%m-%d %H:%M:%S"))))
    
    return candiTweets_total


if __name__ == '__main__':
    # Spark settings
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("TwitterStreamApp")

    # create spark context with the above configuration
    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")
    sql_context = SQLContext(sc)
    ssc = StreamingContext(sc, 5)
    ssc.checkpoint("~/checkpoint_TwitterApp")
    dataStream = ssc.socketTextStream(IP, PORT)
    words = dataStream.flatMap(lambda line: line.split(" "))
    tws = dataStream.flatMap(lambda line: line.split("@"))
    
    filtered_lines = tws.filter(lambda line: any(word in line.lower() for word in WORD))
    filtered_lines.pprint()
    canditw = filtered_lines.map(lambda line: \
                                 (next((word for word in WORD if word.lower() in line.lower()), None), line.lower()))
    candiTweets = canditweet(canditw)
    candiTweets.pprint()
    
    topWords = topWordCount(canditw)
    topWords.pprint()
    
    candiTweets.foreachRDD(lambda rdd: saveToStorage(rdd, output_directory_canditweet, columns_name_canditweet, mode="append"))
    topWords.foreachRDD(lambda rdd: saveToStorage(rdd, output_directory_topwords, columns_name_topwords, mode="append"))
    
    # start the streaming process then wait for streamtime and then stop.
    ssc.start()
    time.sleep(STREAMTIME)
    ssc.stop(stopSparkContext=False, stopGraceFully=True)
    
    saveToBigQuery(sc, output_dataset, output_table_canditweet, output_directory_canditweet)
    saveToBigQuery(sc, output_dataset, output_table_topwords, output_directory_topwords)
    #saveToBucket(candiTweets, bucket_directory)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/02 23:52:47 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/05/02 23:52:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/05/02 23:52:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/05/02 23:52:48 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


-------------------------------------------
Time: 2023-05-02 23:52:55
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:52:55
-------------------------------------------



-------------------------------------------
Time: 2023-05-02 23:52:55
-------------------------------------------



-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------



-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:00
-------------------------------------------



...

-------------------------------------------
Time: 2023-05-02 23:53:45
-------------------------------------------
It always struck me as an option that—inasmuch as it ought to be interp'd as fully w/in the law (i.e. "black letter")— the Court would need to back unless they were wanting to take part of the blame for the econ disaster.House Oversight Chair DISINTEGRATES on LIVE TV When Host Asks for Evidence of Biden Crimes! https://t.co/DkcTvwoVon via 
YouTube  #JamesComer #DollemoreDaily #JoeBiden #RepublicansRT 
SDonziger: My latest column calls out President Biden for caving to the fossil fuel industry as it continues to destroy the planet. In…
SecBlinken told our Senate investigators he did not email Hunter Biden. Not t…RT 
JamesNavaCom: El congresista por Oklahoma Josh Brecheen respalda a Donald Trump para presidente de Estados Unidos. https://t.co/9ADyAXX…
IlhanMN: Fun fact: Republicans voted to raise the debt ceiling 3 times when Donald Trump was President, with no preconditions.

-------------------------------------------
Time: 2023-05-02 23:53:50
-------------------------------------------
LouDobbs: Have you noticed that the Biden Regime is getting away with almost everything they do? Whether it's ordering the southern bor…RT 
DavidSacks: House Republicans have passed a debt ceiling increase but it requires Biden to agree to a 1% annual growth cap, claw back u…RT 
GrillJeff So instead of non-stop Trump coverage there will be non-stop Trump elevation.  Thanks CNN.
MayoIsSpicyy And when did Obama or Biden Serve?  Asking for a friend
1JaySC Ireland 🇮🇪 hates Trump too but loves Biden ! https://t.co/v0Sbskyn8DTrump tops DeSantis by 36 points in new poll https://t.co/F81tT68rvbRT 
TrumpWarRoom: #AGENDA47: President Trump’s plan to reclaim our colleges and universities from radical Marxists and ensure America’s stu…
NickAdamsinUSA LOL Donald "Fascist" Trump would have trouble pressing the bar without any weight on it at all. Trump is behind the deep state, illegal a

-------------------------------------------
Time: 2023-05-02 23:53:50
-------------------------------------------
(('biden', 'struck'), 1)
(('biden', 'option'), 1)
(('biden', 'fully'), 1)
(('biden', '"black'), 1)
(('biden', 'letter")—'), 1)
(('biden', 'court'), 5)
(('biden', 'need'), 2)
(('biden', 'back'), 3)
(('biden', 'unless'), 2)
(('biden', 'part'), 2)
...

-------------------------------------------
Time: 2023-05-02 23:53:50
-------------------------------------------
('biden', ('hunter', 12))
('trump', ('would', 8))

-------------------------------------------
Time: 2023-05-02 23:53:50
-------------------------------------------
('biden', 'hunter', 12)
('trump', 'would', 8)

-------------------------------------------
Time: 2023-05-02 23:53:50
-------------------------------------------
('biden', 'hunter', 12, '2023-05-02 23:53:50')
('trump', 'would', 8, '2023-05-02 23:53:50')

-------------------------------------------
Time: 2023-05-02 23:53:50
---------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:55
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:55
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:53:55
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:54:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:54:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:54:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:54:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:54:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 23:54:00
----------

Waiting on bqjob_r7d2a0d3e9e2ddc52_00000187dee57887_1 ... (2s) Current status: DONE   
Waiting on bqjob_r9a221fbb297e04_00000187dee58e81_1 ... (3s) Current status: DONE   
